esm based dynamic model (not using static embeds).


Use TF:
*  🇰https://colab.research.google.com/github/huggingface/notebooks/blob/main/examples/protein_language_modeling-tf.ipynb#scrollTo=de8419b5
* Torch based /Trainer example:  https://colab.research.google.com/github/huggingface/notebooks/blob/main/examples/protein_language_modeling.ipynb#scrollTo=49dcba23



* NOte for pytorch training could use trainer maybe, and mixed precision - https://huggingface.co/docs/transformers/v4.18.0/en/performance#fp16-training

In [1]:
# from google.colab import drive
# drive.mount('/content/drive')

# # drive.mount("/content/drive/MyDrive/Research/proteins")
# # drive.mount("/content/drive/MyDrive/proteins")

# !pip install tensorflow -U -q
# !pip install torch accelerate transformers fair-esm datasets  -U -q


* Use the unirpot fasta file I downloaded and uploaded to my drive

`/content/drive/MyDrive/Research/biodata/proteins/Transmembrane_human_90.fasta`

* Download fasta from: `https://www.uniprot.org/uniref/?query=uniprot:(keyword%3A%22Transmembrane+%5BKW-0812%5D%22+AND+organism%3A%22Homo+sapiens+%28Human%29+%5B9606%5D%22)+identity:0.9`

In [1]:
# DATA_PATH = "/content/drive/MyDrive/Research/biodata/proteins/membrane_100_whole_protbert_embed.snappy.parquet"
# DATA_PATH = "/content/drive/MyDrive/Research/CIDR-Protein Anomalies project/protein_anomalies_data/swp_human_viri_all_embed_esm.parquet" ## ESM1B embedding (max len 1022)
# DATA_PATH = "/content/drive/MyDrive/proteins/New Protein-Virus anom project/hum_vir_swp-globalEmbed-train.csv.gz"## TRAIN
DATA_PATH = "hum_vir_swp-train.csv.gz"## TRAIN

## TEST data:
# TEST_DATA_PATH = "/content/drive/MyDrive/proteins/New Protein-Virus anom project/hum_vir_swp-globalEmbed-test.csv.gz"## TRAIN
TEST_DATA_PATH = "hum_vir_swp-test.csv.gz"## TEST

# ## metadata for all reviewed/swissprot human + virus proteins
# METADATA_PATH = "/content/drive/MyDrive/Research/CIDR-Protein Anomalies project/protein_anomalies_data/SWP_human_viruses_all.xlsx"

SAVE_INTERMED = False
TARGET_COL = "virus" ## use for filtering data into 1 class

MAX_LEN = 512 # exclude sequences longer than this. (Not merely truncate)

In [2]:
# MY_FASTA_PATH = "/content/drive/MyDrive/Research/biodata/proteins/Transmembrane_human_90.fasta" # 14k sequences, with fragments, 90% id redundnancy

# MY_FASTA_PATH = "/content/drive/MyDrive/Research/biodata/proteins/Transmembrane_human_whole_100.fasta" # 30K sequences, without fragments
# MY_FASTA_PATH = "Transmembrane_human_whole_90.fasta"

# Embed sequences in a FASTA file

In [3]:
# from bio_embeddings.embed import ProtTransBertBFDEmbedder
# from Bio import SeqIO
import torch
import esm
import tensorflow
import tensorflow as tf
from tensorflow.keras.metrics import AUC as tf_auc

import pandas as pd
import numpy as np
import random
# from sklearn.neighbors import LocalOutlierFactor
from sklearn.ensemble import IsolationForest
# from sklearn.svm import OneClassSVM
from sklearn.preprocessing  import MinMaxScaler
import matplotlib.pyplot as plt
from sklearn.metrics import classification_report, roc_auc_score
from sklearn.model_selection import train_test_split, cross_val_predict
from sklearn.linear_model import LogisticRegressionCV, LogisticRegression
# from sklearn.experimental import enable_hist_gradient_boosting
from sklearn.ensemble import HistGradientBoostingClassifier
from tqdm.autonotebook import tqdm

from sklearn.svm import SVC
from sklearn.pipeline import make_pipeline, Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import roc_auc_score
from sklearn.decomposition import PCA

from transformers import TrainingArguments, Trainer, logging
from accelerate import Accelerator
from torch.utils.data.dataloader import DataLoader
## https://huggingface.co/docs/transformers/perf_train_gpu_one

2023-08-29 12:51:53.020776: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-08-29 12:51:53.042206: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI AVX512_BF16 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-08-29 12:51:53.434166: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
/tmp/ipykernel_14647/3407002226.py:22: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm


In [4]:
def auc(y_true, y_pred):
    auc = tensorflow.metrics.auc(y_true, y_pred)[1]
    K.get_session().run(tf.local_variables_initializer())
    return auc

In [5]:
# from tensorflow import keras
# from tensorflow.keras import layers
# from tensorflow.keras import mixed_precision
# policy = mixed_precision.Policy('mixed_float16')
# mixed_precision.set_global_policy(policy)
# tensorflow.keras.mixed_precision.set_global_policy('mixed_float16') # causes error with our pretrained model

In [6]:
# embedder = ProtTransBertBFDEmbedder(half_model=True)

In [15]:
# df = pd.read_parquet(DATA_PATH) # numpy to pandas
df = pd.read_csv(DATA_PATH,usecols=["Sequence","virus","Length","Cluster name"])
df_test = pd.read_csv(TEST_DATA_PATH,usecols=["Sequence","virus","Length","Cluster name"])
 ## lengths of all the seqs
    
df["Cluster name"] = df["Cluster name"].str.replace("Cluster:","").str.strip()
df_test["Cluster name"] = df_test["Cluster name"].str.replace("Cluster:","").str.strip()

df

,Sequence,virus,Length,Cluster name
0,MADFLKGLPVYNKSNFSRFHADSVCKASNRRPSVYLPTREYPSEQI...,0,102,DET1- and DDB1-associated protein 1
1,MPSTLQVLAKKVLALEHKENDHISREYYYHILKCCGLWWHEAPIIL...,1,362,Protein MGF 360-19R
2,MASSAELDFNLQALLEQLSQDELSKFKSLIRTISLGKELQTVPQTE...,0,97,Pyrin domain-containing protein 2
3,MAAWGKKHAGKDPVRDECEERNRFTETREEDVTDEHGEREPFAETD...,0,186,Protein FAM9B
4,MASDSPARSLDEIDLSALRDPAGIFELVELVGNGTYGQVYKGRHVK...,0,1360,TRAF2 and NCK-interacting protein kinase
...,...,...,...,...
20335,MDPDKQDALNSIENSIYRTAFKLQSVQTLCQLDLIDSSLIQQVLLR...,0,578,Dystrotelin
20336,MLCPWRTANLGLLLILTIFLVAEAEGAAQPNNSLMLQTSKENHALA...,0,348,Cell surface glycoprotein CD200 receptor 1
20337,MCLRFFSPVPGSTSSATNVTMVVSAGPWSSEKAEMNILEINEKLRP...,0,421,Putative neuroblastoma breakpoint family member 7
20338,MASHAGQQHAPAFGQAARASGPTDGRAASRPSHRQGASEARGDPEL...,1,376,Thymidine kinase


In [16]:
df.groupby(["virus"])["Length"].describe().round(2)

,count,mean,std,min,25%,50%,75%,max
virus,,,,,,,,
0,14668.0,473.16,308.52,11.0,244.0,398.0,623.0,1533.0
1,5672.0,380.31,298.35,11.0,151.0,297.0,514.0,1520.0


In [17]:
print(df["virus"].agg(["mean","sum","count"]).round(2))
# df["Length"].describe().round(1)

mean         0.28
sum       5672.00
count    20340.00
Name: virus, dtype: float64


In [18]:
df_test["Length"].describe().round(1)

count    4777.0
mean      444.3
std       304.5
min        18.0
25%       212.0
50%       369.0
75%       589.0
max      1534.0
Name: Length, dtype: float64

In [19]:
df = df.loc[df["Length"]<=MAX_LEN].reset_index(drop=True)
df_test = df_test.loc[df_test["Length"]<=MAX_LEN].reset_index(drop=True)

print(df["Length"].describe().round(1))
print(df["virus"].agg(["mean","sum","count"]).round(2))

count    19001.0
mean       392.2
std        233.5
min         11.0
25%        204.0
50%        350.0
75%        535.0
max       1024.0
Name: Length, dtype: float64
mean         0.28
sum       5414.00
count    19001.00
Name: virus, dtype: float64


In [20]:

if SAVE_INTERMED:
    ## save smaller file to disk/drive
    df1 = df.loc[df["Length"]<=1024].reset_index(drop=True)
    df_test1 = df_test.loc[df_test["Length"]<=1024].reset_index(drop=True)

    print(df_test1.shape[0],"test 1 <1024")
    display(df1)

    df1.to_csv("hum_vir_swp-train.csv.gz",index=False,compression="gzip")
    df_test1.to_csv("hum_vir_swp-test.csv.gz",index=False,compression="gzip")


4491 test 1 <1024


In [13]:
# ## metadata about all sequences, can be used to identify and to define targets/labels
# df_meta = pd.read_excel(METADATA_PATH).dropna(how="all",axis=1)
# df_meta

### example pretrained fb/torch:
* https://github.com/facebookresearch/esm#getting-started-with-this-repo-

* Transformers + trainer example : (mlm case): https://github.com/facebookresearch/esm/discussions/556
* keras models supported / via HF?
  * https://huggingface.co/docs/transformers/model_doc/esm#transformers.EsmForSequenceClassification.forward.example


  TF finetuning example (sequence evel?):
  https://colab.research.google.com/github/huggingface/notebooks/blob/main/examples/protein_language_modeling-tf.ipynb#scrollTo=4b26b828

In [14]:
from sklearn.model_selection import train_test_split
from datasets import Dataset
from transformers import AdamWeightDecay
from tensorflow.keras.optimizers import Adafactor, Adam # more memory effecient than adamWD
import tensorflow
# from tensorflow.keras.metrics.AUC()
# from transformers import AutoTokenizer #DataCollatorForLanguageModeling,
## https://huggingface.co/docs/transformers/model_doc/esm#transformers.EsmForSequenceClassification.forward.example
from transformers import AutoTokenizer, EsmForSequenceClassification, Trainer
from transformers import TFAutoModelForTokenClassification, TFAutoModelForSequenceClassification ,TFEsmForSequenceClassification

In [15]:
# sequences = df["Sequence"].tolist()
# labels = df["virus"].tolist()
# groups = df["Cluster name"].tolist()

# assert len(sequences) == len(labels) # Quick check to make sure we got it right

# train_sequences, test_sequences, train_labels, test_labels = train_test_split(sequences, labels, test_size=0.25, shuffle=True)#,stratify=labels)

In [16]:
train_sequences = df["Sequence"].tolist()
train_labels = df["virus"].tolist()
train_groups = df["Cluster name"].tolist()

# train_sequences, test_sequences, train_labels, test_labels = train_test_split(sequences, labels, test_size=0.25, shuffle=True)#,stratify=labels)

test_sequences = df_test["Sequence"].tolist()
test_labels = df_test["virus"].tolist()

In [17]:
# Load ESM-2 model
## smallest: esm2_t6_8M_UR50D
## 2d smallest
## large: esm2_t33_650M_UR50D
# model_checkpoint =  "facebook/esm2_t12_35M_UR50D"#"facebook/esm2_t6_8M_UR50D"
model_checkpoint =  "facebook/esm2_t30_150M_UR50D"
 # ElnaggarLab/ankh-base
 ### https://github.com/agemagician/Ankh/blob/main/examples/binary_classification_solubility_task.ipynb - different model?
 #  https://github.com/agemagician/Ankh#models   - 450M model size
# model_checkpoint =   "ElnaggarLab/ankh-base"

model_max_len = min(800,MAX_LEN)

tokenizer = AutoTokenizer.from_pretrained(model_checkpoint, padding=True, truncation=True,max_length=model_max_len)

num_labels = max(train_labels + test_labels) #+ 1  # Add 1 since 0 can be a label
print("Num labels:", num_labels)
model = TFAutoModelForSequenceClassification.from_pretrained(model_checkpoint, num_labels=num_labels,problem_type="single_label_classification") # worked, orig
# model = EsmForSequenceClassification.from_pretrained(model_checkpoint, num_labels=num_labels)
# model = TFEsmForSequenceClassification.from_pretrained(model_checkpoint, num_labels=num_labels,problem_type="single_label_classification")


# model.eval()  # disables dropout for deterministic results # from original - in infer/embed only example

# last_layer_num = model.num_layers ## 33 for esm2_t33_650M_UR50D
# print(last_layer_num )

Num labels: 1


2023-08-29 10:31:15.872919: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:982] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-08-29 10:31:15.873433: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:982] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-08-29 10:31:15.873449: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:982] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-08-29 10:31:15.874522: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:982] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-08-29 10:31:15.874540: I tensorflow/compile

In [18]:
model.config

EsmConfig {
  "_name_or_path": "facebook/esm2_t30_150M_UR50D",
  "architectures": [
    "EsmForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.0,
  "classifier_dropout": null,
  "emb_layer_norm_before": false,
  "esmfold_config": null,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.0,
  "hidden_size": 640,
  "id2label": {
    "0": "LABEL_0"
  },
  "initializer_range": 0.02,
  "intermediate_size": 2560,
  "is_folding_model": false,
  "label2id": {
    "LABEL_0": 0
  },
  "layer_norm_eps": 1e-05,
  "mask_token_id": 32,
  "max_position_embeddings": 1026,
  "model_type": "esm",
  "num_attention_heads": 20,
  "num_hidden_layers": 30,
  "pad_token_id": 1,
  "position_embedding_type": "rotary",
  "problem_type": "single_label_classification",
  "token_dropout": true,
  "torch_dtype": "float32",
  "transformers_version": "4.31.0",
  "use_cache": true,
  "vocab_list": null,
  "vocab_size": 33
}

In [19]:
## Warning - longer than allowed length - 1024
train_tokenized = tokenizer( train_sequences,  truncation=True,max_length=model_max_len,padding=True) # padding=True,
test_tokenized = tokenizer(test_sequences,  truncation=True,max_length=model_max_len,padding=True) # padding=True,

In [20]:
train_dataset = Dataset.from_dict(train_tokenized)
test_dataset = Dataset.from_dict(test_tokenized)

train_dataset

Dataset({
    features: ['input_ids', 'attention_mask'],
    num_rows: 13727
})

In [21]:
train_dataset = train_dataset.add_column("labels", train_labels)
test_dataset = test_dataset.add_column("labels", test_labels)
train_dataset

Dataset({
    features: ['input_ids', 'attention_mask', 'labels'],
    num_rows: 13727
})

In [22]:
tf_train_set = model.prepare_tf_dataset(
    train_dataset,
    batch_size= 5
    ,shuffle=True,
    tokenizer=tokenizer
)

tf_test_set = model.prepare_tf_dataset(
    test_dataset,
    batch_size=5
    ,shuffle=False,
    tokenizer=tokenizer
)

Compile keras model

In [23]:
# Freeze all layers except the last few
for layer in model.layers[3:-8]:
    layer.trainable = False

In [24]:

# opt = Adafactor(7e-5)##AdamWeightDecay(1e-4) #default: AdamWeightDecay(2e-5)
opt = Adam(7e-5)
# opt = AdamWeightDecay(7e-5)
# loss = tf.keras.losses.BinaryCrossentropy(from_logits=True)
#
model.compile(optimizer=opt, metrics=["accuracy",tf.keras.metrics.AUC(num_thresholds=20,from_logits=True)], # ,tf.keras.metrics.Precision() - fails, multiclass/shape? same for auc
              loss= "BinaryFocalCrossentropy" #"BinaryCrossentropy"
              ) ## "BinaryFocalCrossentropy" - seems worse, but actual eval metrics look better!

# # Reports the AUC of a model outputting a probability.
# model.compile(optimizer=opt, #'sgd',
#               loss=tensorflow.keras.losses.BinaryCrossentropy(),
#               metrics=[tensorflow.keras.metrics.AUC()])

# # Reports the AUC of a model outputting a logit.
# model.compile(optimizer=opt,
#               loss=tensorflow.keras.losses.BinaryCrossentropy(from_logits=True),
#               metrics=[tf.keras.metrics.AUC(from_logits=True)])

In [25]:
model.fit(tf_train_set, validation_data=tf_test_set, epochs=7)

Epoch 1/5


2023-08-29 10:31:42.564794: I tensorflow/compiler/xla/service/service.cc:169] XLA service 0x7f66c012ce20 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2023-08-29 10:31:42.564819: I tensorflow/compiler/xla/service/service.cc:177]   StreamExecutor device (0): NVIDIA GeForce RTX 4080, Compute Capability 8.9
2023-08-29 10:31:42.567047: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:269] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2023-08-29 10:31:42.641779: I tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:424] Loaded cuDNN version 8700
2023-08-29 10:31:42.706248: I tensorflow/tsl/platform/default/subprocess.cc:304] Start cannot spawn child process: Permission denied
2023-08-29 10:31:42.749212: I ./tensorflow/compiler/jit/device_compiler.h:180] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


3431/3431 [==============================] - 843s 234ms/step - loss: 0.0889 - accuracy: 0.8648 - auc: 0.9037 - val_loss: 0.0589 - val_accuracy: 0.9279 - val_auc: 0.9622
Epoch 2/5
3431/3431 [==============================] - 798s 232ms/step - loss: 0.0525 - accuracy: 0.9272 - auc: 0.9651 - val_loss: 0.0598 - val_accuracy: 0.9223 - val_auc: 0.9694
Epoch 3/5
3431/3431 [==============================] - 795s 232ms/step - loss: 0.0345 - accuracy: 0.9593 - auc: 0.9854 - val_loss: 0.0422 - val_accuracy: 0.9400 - val_auc: 0.9717
Epoch 4/5
3431/3431 [==============================] - 794s 231ms/step - loss: 0.0302 - accuracy: 0.9671 - auc: 0.9899 - val_loss: 0.0755 - val_accuracy: 0.9393 - val_auc: 0.9698
Epoch 5/5
3431/3431 [==============================] - 793s 231ms/step - loss: 0.0199 - accuracy: 0.9817 - auc: 0.9957 - val_loss: 0.0682 - val_accuracy: 0.9344 - val_auc: 0.9687


With 150M model. Max 512 length (drop datapoints).

```
model.layers[3:-8]
"BinaryFocalCrossentropy" 
opt = Adam(7e-5)

3431/3431 [==============================] - 843s 234ms/step - loss: 0.0889 - accuracy: 0.8648 - auc: 0.9037 - val_loss: 0.0589 - val_accuracy: 0.9279 - val_auc: 0.9622
Epoch 2/5
3431/3431 [==============================] - 798s 232ms/step - loss: 0.0525 - accuracy: 0.9272 - auc: 0.9651 - val_loss: 0.0598 - val_accuracy: 0.9223 - val_auc: 0.9694
Epoch 3/5
3431/3431 [==============================] - 795s 232ms/step - loss: 0.0345 - accuracy: 0.9593 - auc: 0.9854 - val_loss: 0.0422 - val_accuracy: 0.9400 - val_auc: 0.9717
Epoch 4/5
3431/3431 [==============================] - 794s 231ms/step - loss: 0.0302 - accuracy: 0.9671 - auc: 0.9899 - val_loss: 0.0755 - val_accuracy: 0.9393 - val_auc: 0.9698
Epoch 5/5
3431/3431 [==============================] - 793s 231ms/step - loss: 0.0199 - accuracy: 0.9817 - auc: 0.9957 - val_loss: 0.0682 - val_accuracy: 0.9344 - val_auc: 0.9687
```


With 35M model :
```
Default (1024) max length
batch_size=8

opt = Adafactor(1e-4)##AdamWeightDecay(1e-4) #default: AdamWeightDecay(2e-5)
model.compile(optimizer=opt, metrics=["accuracy"],
              loss="BinaryCrossentropy")
3813/3813 [==============================] - 2625s 661ms/step - loss: 0.2452 - accuracy: 0.9104 - val_loss: 0.1622 - val_accuracy: 0.9363
Epoch 2/3
3813/3813 [==============================] - 2518s 661ms/step - loss: 0.1218 - accuracy: 0.9597 - val_loss: 0.1533 - val_accuracy: 0.9463
Epoch 3/3
3813/3813 [==============================] - 2523s 662ms/step - loss: 0.0730 - accuracy: 0.9799 - val_loss: 0.1978 - val_accuracy: 0.9436

```

In [26]:
# model.save("trained_esm.keras")
model.save("trained_esm.tf",save_format="tf")

2023-08-29 11:38:31.977038: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'attention_mask' with dtype int32 and shape [?,?]
	 [[{{node attention_mask}}]]
2023-08-29 11:38:34.641171: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'attention_mask' with dtype int32 and shape [?,?]
	 [[{{node attention_mask}}]]
2023-08-29 11:38:34.822747: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'input_ids_1' with dtype int32 and shape [?,?]
	 [[{{n

INFO:tensorflow:Assets written to: trained_esm.tf/assets


INFO:tensorflow:Assets written to: trained_esm.tf/assets


In [27]:
y_test_pred = model.predict(tf_test_set)

808/808 [==============================] - 102s 121ms/step


In [29]:
# print(classification_report(test_labels,y_test_pred.logits[:,1]>=0.5))
print(classification_report(test_labels,y_test_pred.logits>=0.5))

              precision    recall  f1-score   support

           0       0.96      0.96      0.96      2448
           1       0.86      0.86      0.86       783

    accuracy                           0.93      3231
   macro avg       0.91      0.91      0.91      3231
weighted avg       0.93      0.93      0.93      3231



In [30]:
roc_auc_score(test_labels,y_test_pred.logits)

0.9782302022554445

In [33]:
df_test

,Sequence,virus,Length,Cluster name
0,MTTATPLGDTTFFSLNMTTRGEDFLYKSSGAIVAAVVVVVIIIFTV...,0,102,Cluster: Noncompact myelin-associated protein
1,MGTAAAAAAAAAAAAAGEGARSPSPAAVSLGLGVAVVSSLVNGSTF...,0,329,Cluster: Magnesium transporter NIPA1
2,MFTIKLLLFIVPLVISSRIDQDNSSFDSLSPEPKSRFAMLDDVKIL...,0,460,Cluster: Angiopoietin-related protein 3
3,MPRGQKSKLRAREKRRKAREETQGLKVAHATAAEKEECPSSSPVLG...,0,347,Cluster: Melanoma-associated antigen B1
4,MIYTEFTILYKSSLPKYLRIMYFTSIGIFFHKLSIVSLEIVSSKTV...,1,117,Cluster: Uncharacterized protein FPV220
...,...,...,...,...
3226,MVCEKCEKKLGTVITPDTWKDGARNTTESGGRKLNENKALTSKKAR...,0,101,Cluster: Cysteine-rich PDZ-binding protein
3227,MAAGDGDVKLGTLGSGSESSNDGGSESPGDAGAAAEGGGWAAAALA...,0,223,Cluster: Membrane-associated progesterone rece...
3228,MTLPASLKGFETLDIFNISSVRNICELNKHAEHVSIKNLPLIDISV...,1,256,Cluster: Cytoplasmic envelopment protein 1
3229,MKWKALFTAAILQAQLPITEAQSFGLLDPKLCYLLDGILFIYGVIL...,0,164,Cluster: T-cell surface glycoprotein CD3 zeta ...


In [35]:
len(test_labels)

3231

In [38]:
df_test.shape[0]

3231

In [42]:
len(y_test_pred.logits>=0.5)

3231

In [50]:
y_test_binary_preds = (y_test_pred.logits>=0.5).astype(int).ravel()
print(len(y_test_binary_preds))

3231


In [54]:
test_mistakes_mask = test_labels!=y_test_binary_preds
print(test_mistakes_mask.sum())
print(len(test_mistakes_mask))

212
3231


In [56]:
df_test[test_mistakes_mask].drop_duplicates(subset="Cluster name")

,Sequence,virus,Length,Cluster name
5,MHGRAYLLLHRDFCDLKENNYKGITAKPVSEDMMEWEVEIEGLQNS...,0,321,Cluster: Ubiquitin-conjugating enzyme E2 U
36,MTKTALLKLFVAIVITFILILPEYFKTPKERTLELSCLEVCLQSNF...,0,296,Cluster: Transmembrane protein 156
44,MATVTAPMNMWSVRIVNGIFVLSNKKPLNLPAHLSQLNIPYSVHFI...,1,404,Cluster: Uncharacterized gene 10 protein
80,MPPKGNKQAIADRRSQKQQKLQEQWDEEEESWDDSQAEEVSDEEEM...,1,217,Cluster: Protein 33K
89,MRSSTLQDPRRRDPQDDVYVDITDRLRFAILYSRPKSASNVHYFSI...,0,447,Cluster: Dual specificity protein phosphatase ...
...,...,...,...,...
3147,MDTQKDVQPPKQQPMIYICGECHTENEIKSRDPIRCRECGYRIMYK...,0,58,"Cluster: DNA-directed RNA polymerases I, II, a..."
3167,MKKDKKRNKIPIDNYPIQTLVNMSLNPSRPSSSELVELHVFYVPEG...,0,459,Cluster: Spermatogenesis-associated protein 1
3180,MAPPSRHCLLLISTLGVFALNCFTKGQKNSTLIFTRENTIRNCSCS...,0,219,Cluster: Uncharacterized protein C21orf62
3186,MYKCSQGAMNTEKVMEKFVIQSRFREMYPDKAKAIAGMTVPARYAD...,1,90,Cluster: Uncharacterized protein 045R


In [ ]:
# ### AttributeError: 'TFEsmForSequenceClassification' object has no attribute 'to'

# training_args = TrainingArguments(
#     per_device_train_batch_size=4,
#     gradient_accumulation_steps=4,
#     # gradient_checkpointing=True,
#     fp16=True,
#     optim="adafactor",

#     evaluation_strategy="steps",
#     warmup_ratio=0.1,
#     save_strategy="no",
#     eval_steps=1000,
#     # **default_args,
#     output_dir="."
# )
# #eval_dataset =
# trainer = Trainer(model=model, args=training_args, train_dataset=tf_train_set,tokenizer=tokenizer,)
# result = trainer.train()
# print_summary(result)



In [ ]:
# model, alphabet = esm.pretrained.esm2_t6_8M_UR50D()
# batch_converter = alphabet.get_batch_converter()

# # Prepare data (first 2 sequences from ESMStructuralSplitDataset superfamily / 4)
# data = [
#     ("protein1", "MKTVRQERLKSIVRILERSKEPVSGAQLAEELSVSRQVIVQDIAYLRSLGYNIVATPRGYVLAGG"),
#     ("protein2", "KALTARQQEVFDLIRDHISQTGMPPTRAEIAQRLGFRSPNAAEEHLKALARKGVIEIVSGASRGIRLLQEE"),
#     ("protein2 with mask","KALTARQQEVFDLIRD<mask>ISQTGMPPTRAEIAQRLGFRSPNAAEEHLKALARKGVIEIVSGASRGIRLLQEE"),
#     ("protein3",  "K A <mask> I S Q"),
# ]
# batch_labels, batch_strs, batch_tokens = batch_converter(data)
# batch_lens = (batch_tokens != alphabet.padding_idx).sum(1)

# # Extract per-residue representations (on CPU)
# with torch.no_grad():
#     results = model(batch_tokens, repr_layers=[last_layer_num], return_contacts=True)
# token_representations = results["representations"][last_layer_num]

# # Generate per-sequence representations via averaging
# # NOTE: token 0 is always a beginning-of-sequence token, so the first residue is token 1.
# sequence_representations = []
# for i, tokens_len in enumerate(batch_lens):
#     sequence_representations.append(token_representations[i, 1 : tokens_len - 1].mean(0))

# # Look at the unsupervised self-attention map contact predictions

# # for (_, seq), tokens_len, attention_contacts in zip(data, batch_lens, results["contacts"]):
# #     plt.matshow(attention_contacts[: tokens_len, : tokens_len])
# #     plt.title(seq)
# #     plt.show()

* https://huggingface.co/docs/transformers/model_doc/esm#transformers.EsmForSequenceClassification.forward.example

In [ ]:
# import torch
# from transformers import AutoTokenizer, EsmForSequenceClassification

# tokenizer = AutoTokenizer.from_pretrained("facebook/esm2_t6_8M_UR50D")
# model = EsmForSequenceClassification.from_pretrained("facebook/esm2_t6_8M_UR50D")

# inputs = tokenizer("Hello, my dog is cute", return_tensors="pt")

# with torch.no_grad():
#     logits = model(**inputs).logits

# predicted_class_id = logits.argmax().item()

# # To train a model on `num_labels` classes, you can pass `num_labels=num_labels` to `.from_pretrained(...)`
# num_labels = len(model.config.id2label)
# model = EsmForSequenceClassification.from_pretrained("facebook/esm2_t6_8M_UR50D", num_labels=num_labels)

# labels = torch.tensor([1])
# loss = model(**inputs, labels=labels).loss